# GETTIN
## Planejamento de Equipamentos Escolares
## Método: Capacitaded Facility Location
### Versão: 0.9
#### Fillipe O Feitosa <fillipefeitosa@ua.pt>

---


#### Math Programming


\begin{split}& \text{minimizar} \quad    & \sum_{j=1}^m f_j x_j +\sum_{i=1}^n \sum_{j=1}^m c_{ij} y_{ij} &     \\
& \text{sujeito a:} \quad & \sum_{j=1}^m y_{ij} =d_i  &  \mbox{ for }  i=1,\cdots,n \\
&    & \sum_{i=1}^n y_{ij} \leq M_j x_j & \mbox{ para } j=1,\cdots,m  \\
&    & y_{ij} \leq d_i x_j              & \mbox{ para } i=1,\cdots,n; j=1,\cdots,m \\
&    & y_{ij} \geq 0                    & \mbox{ para } i=1,\cdots,n; j=1,\cdots,m \\
&    & x_j \in \{ 0,1 \}                & \mbox{ para } j=1,\cdots,m\end{split}

#### List of Imports

In [1]:
# Import Libraries
from gurobi import *
import math
import matplotlib.pyplot as plt
import pandas as pd
import geopandas as gpd
import geopy.distance
from collections import defaultdict
# TEST Used for testing with a small sample
import random


#### Problem Data

In [2]:
# ------------------------- Problem data ------------------------- #

# Dict for SubSections With Demand
rawData = pd.read_excel('./subseccao_vagos/Pop_escolar_Vagos.xlsx')
listOfSubsections_2010 = {}
for e in range(len(rawData)-1):
    listOfSubsections_2010[int(rawData.SUBSECCAO[e])] = rawData.P_esc_1CEB_2010[e]+ rawData.P_esc_2CEB_2010[e]+rawData.P_esc_3CEB_2010[e]+rawData.P_esc_secund_2010[e]+rawData.Pre_escolar_2010[e]

    
# TEST Used _sample for testing with a small sample to CREATE A SAMPLE OF POTENTIAL SCHOOL POINTS   
listOfSubsections_2010_sample = {}

for i in range(30):
    sampleSchool, sampleDemand = random.choice(list(listOfSubsections_2010.items()))
    # If statement: Do not repeat random schools
    if sampleSchool not in listOfSubsections_2010_sample.keys():
        listOfSubsections_2010_sample[sampleSchool] = sampleDemand
        

# Dict of Lists for Schools with Capacity And Cost by Size
tupleOfCapacity = (80, 120, 200)

# TEST used for testing with small sample
# tupleOfCapacity = (10, 20, 50)

schoolsCapacityCost = defaultdict(list)

# # TEST Using _SAMPLE to get a sample of Points to create schools
for bgri in list(listOfSubsections_2010_sample.keys()):
    for capacity in tupleOfCapacity:
        cost = capacity*6200
        tempList = (capacity, cost)
        schoolsCapacityCost[bgri].append(tempList)


# Retrieve Coordinates and Vinculate to Identifier (BGRI)
vagos = gpd.read_file('./data_gettin/vagos.geojson');
centroids = vagos.centroid

iteratorHandler = centroids.size
centroidVector = []
for centroid in centroids:
    obj = [centroid.xy[0][0], centroid.xy[1][0]]
    centroidVector.append(obj)
    
coordinates_X = {}
coordinates_Y = {}
for e in range (len(vagos)):
    coordinates_X[int(vagos.BGRI11[e])] = centroids[e].xy[0][0]
    coordinates_Y[int(vagos.BGRI11[e])] = centroids[e].xy[1][0]
    
coordinates_X[list(listOfSubsections_2010.keys())[0]]

-8.69434885302111

In [3]:
if 1180400403 not in listOfSubsections_2010_sample.keys():
    print("nao está")
sum(listOfSubsections_2010.values())

nao está


4034.000000000001

#### Modeling and creation of decision Variables

In [4]:
   
# -------- Distance Matrix Logic ------- #
transportationCosts = {}

for bgri_1 in list(listOfSubsections_2010.keys()):
    coords_1 = (coordinates_X[bgri_1], coordinates_Y[bgri_1])
    for bgri_2 in list(listOfSubsections_2010.keys()):
        coords_2 = (coordinates_X[bgri_2], coordinates_Y[bgri_2])
        # 0.29 cents per KM * 180 days
        cost = geopy.distance.vincenty(coords_1, coords_2).km*0.29*180*5
        transportationCosts[bgri_1, bgri_2] = cost
        
transportationCosts


{(1180100101, 1180100101): 0.0,
 (1180100101, 1180100102): 53.85232201410466,
 (1180100101, 1180100103): 75.70593525830172,
 (1180100101, 1180100104): 189.63754020712878,
 (1180100101, 1180100105): 124.33378869616459,
 (1180100101, 1180100106): 259.27879343375247,
 (1180100101, 1180100107): 322.6716804723019,
 (1180100101, 1180100108): 308.2463745258148,
 (1180100101, 1180100109): 424.6501600078313,
 (1180100101, 1180100110): 334.9931421300826,
 (1180100101, 1180100111): 375.2085475324842,
 (1180100101, 1180100112): 401.0625134054475,
 (1180100101, 1180100113): 380.2706810688313,
 (1180100101, 1180100114): 429.24666337842524,
 (1180100101, 1180100115): 572.6506288846633,
 (1180100101, 1180100116): 536.5653861795611,
 (1180100101, 1180100117): 189.1663824102672,
 (1180100101, 1180100118): 329.9827925739894,
 (1180100101, 1180100119): 270.9734316524683,
 (1180100101, 1180100120): 408.61721500237127,
 (1180100101, 1180100121): 274.19160105050986,
 (1180100101, 1180100122): 346.88097983249

In [5]:
# for school in list(schoolsCapacityCost.keys()):
#     for schoolSize in range(len(schoolsCapacityCost[school])):
#         print(schoolsCapacityCost[school][schoolSize][0])
#         print(schoolSize)

In [6]:

# -------- Decision Variables ---------- #

# numSchools = len(escola)
# numSubSections = len(subSecao)

# Creting Guroby Model
m = Model()

# Decision Variables
x = {}
y = {}
# d = {} # Distance Matrix
# @alpha: 0.29 de custo por Km  por (365 dias * 5 anos) 
# alpha = 529.25

# creating binary variable for every school
for j in list(schoolsCapacityCost.keys()):
    for schoolSize in tupleOfCapacity:
        x[(j,schoolSize)] = m.addVar(vtype=GRB.BINARY, name="escola(%d,%d)" % (j, schoolSize))

# creating continuous variable for subsections to check suply fractions
for subsection in list(listOfSubsections_2010.keys()):
    for school in list(schoolsCapacityCost.keys()):
        for capacity in schoolsCapacityCost[school]:
        # y of Subsection Suply
            y[(subsection,school,capacity[0])] = m.addVar(lb=0, vtype=GRB.CONTINUOUS, name="Fração da Sub[%d], escola[%d][%d]" % (subsection,school, capacity[0]))

m.update()

In [7]:
# schoolsCapacityCost[1180100101][1]
# listOfSubsections_2010[1180100101]
# for school in list(schoolsCapacityCost.keys()):
#     for schoolSize in range(len(schoolsCapacityCost[school])):
#         sizeOfSchool = schoolsCapacityCost[school][schoolSize][0]
#         print(x[school, sizeOfSchool])
# for capacity in schoolsCapacityCost[1180800306]:
#     print(capacity[0])

for capacity in tupleOfCapacity:
    print(capacity)

80
120
200


## Adding Constraints

In [8]:
# Constraint for Every Student on School
# for i in range(numSubSections):
#     m.addConstr(quicksum(y[(i,j)] for j in range(numSchools)) == 1)

# Constraint to repect demands on Every Subsection - OK
for i in list(listOfSubsections_2010.keys()):
    m.addConstr(quicksum(y[(i,j,k)] for j in list(schoolsCapacityCost.keys()) for k in tupleOfCapacity) == listOfSubsections_2010[i])
    
# Constraint to restrain school capacity - OK
for school in list(schoolsCapacityCost.keys()):
    for schoolSize in range(len(schoolsCapacityCost[school])):
        numericalSizeOfSchool = schoolsCapacityCost[school][schoolSize][0]
        m.addConstr(quicksum(y[(i,school, numericalSizeOfSchool)] for i in list(listOfSubsections_2010.keys())) <= numericalSizeOfSchool * x[school, numericalSizeOfSchool], "SchoolK(%s,%s)"%(school, numericalSizeOfSchool))
    
# Add restriction to fraction Size - OK
for (i,j,k) in y:
    m.addConstr(y[i,j,k] <= listOfSubsections_2010[i]*x[j, k], "FractionConst(%s,%s,%s)"%(i,j,k))

#### Objetive


In [9]:
# Setting objective

m.setObjective(
    quicksum(schoolsCapacityCost[school][schoolSize][1]*x[school, schoolsCapacityCost[school][schoolSize][0]] for school in list(schoolsCapacityCost.keys()) for schoolSize in range(len(schoolsCapacityCost[school]))) + 
    quicksum(transportationCosts[i,j]*y[i,j,k] for i in list(listOfSubsections_2010.keys()) for j in list(schoolsCapacityCost.keys()) for k in tupleOfCapacity), GRB.MINIMIZE )



In [10]:
m.optimize()

Optimize a model with 58148 rows, 57510 columns and 225180 nonzeros
Variable types: 57420 continuous, 90 integer (90 binary)
Coefficient statistics:
  Matrix range     [3e-01, 2e+02]
  Objective range  [2e+01, 1e+06]
  Bounds range     [1e+00, 1e+00]
  RHS range        [3e-01, 5e+01]
Found heuristic solution: objective 8.112129e+07
Presolve removed 4641 rows and 4590 columns
Presolve time: 0.36s
Presolved: 53507 rows, 52920 columns, 211410 nonzeros
Variable types: 52830 continuous, 90 integer (90 binary)

Deterministic concurrent LP optimizer: primal and dual simplex
Showing first log only...

Presolved: 53507 rows, 52920 columns, 211410 nonzeros

Presolve removed 52830 rows and 90 columns
Concurrent spin time: 0.36s

Solved with dual simplex

Root relaxation: objective 2.612803e+07, 18126 iterations, 4.84 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 2.6128e+0

In [11]:
m.getConstrs()

[<gurobi.Constr R0>,
 <gurobi.Constr R1>,
 <gurobi.Constr R2>,
 <gurobi.Constr R3>,
 <gurobi.Constr R4>,
 <gurobi.Constr R5>,
 <gurobi.Constr R6>,
 <gurobi.Constr R7>,
 <gurobi.Constr R8>,
 <gurobi.Constr R9>,
 <gurobi.Constr R10>,
 <gurobi.Constr R11>,
 <gurobi.Constr R12>,
 <gurobi.Constr R13>,
 <gurobi.Constr R14>,
 <gurobi.Constr R15>,
 <gurobi.Constr R16>,
 <gurobi.Constr R17>,
 <gurobi.Constr R18>,
 <gurobi.Constr R19>,
 <gurobi.Constr R20>,
 <gurobi.Constr R21>,
 <gurobi.Constr R22>,
 <gurobi.Constr R23>,
 <gurobi.Constr R24>,
 <gurobi.Constr R25>,
 <gurobi.Constr R26>,
 <gurobi.Constr R27>,
 <gurobi.Constr R28>,
 <gurobi.Constr R29>,
 <gurobi.Constr R30>,
 <gurobi.Constr R31>,
 <gurobi.Constr R32>,
 <gurobi.Constr R33>,
 <gurobi.Constr R34>,
 <gurobi.Constr R35>,
 <gurobi.Constr R36>,
 <gurobi.Constr R37>,
 <gurobi.Constr R38>,
 <gurobi.Constr R39>,
 <gurobi.Constr R40>,
 <gurobi.Constr R41>,
 <gurobi.Constr R42>,
 <gurobi.Constr R43>,
 <gurobi.Constr R44>,
 <gurobi.Constr R45>

In [12]:
print('Obj: %g' % m.objVal)

Obj: 2.61859e+07


In [13]:
resultsBgriCapacity = []
resultsSubFractionToSchool = []
for v in m.getVars():
    if(v.x != 0):
        print('%s   %g' % (v.varName, v.x))
        if(v.varName.startswith('escola')):
            resultsBgriCapacity.append(v.varName)        
        else:
            resultsSubFractionToSchool.append(v.varName)

escola(1180600121,80)   1
escola(1180600121,120)   1
escola(1180400104,120)   1
escola(1181100114,120)   1
escola(1180700110,120)   1
escola(1180300206,80)   1
escola(1180300206,120)   1
escola(1180200202,80)   1
escola(1180800605,80)   1
escola(1180800605,120)   1
escola(1180800605,200)   1
escola(1180100209,80)   1
escola(1180100209,120)   1
escola(1180400606,80)   1
escola(1180400210,80)   1
escola(1180400210,200)   1
escola(1180100112,80)   1
escola(1180100112,120)   1
escola(1180700304,120)   1
escola(1180500217,80)   1
escola(1180700115,120)   1
escola(1181000302,80)   1
escola(1181000302,120)   1
escola(1181000302,200)   1
escola(1180200225,80)   1
escola(1180600120,120)   1
escola(1180800607,80)   1
escola(1180800607,120)   1
escola(1180800607,200)   1
escola(1180700307,80)   1
escola(1180400603,80)   1
escola(1180700412,120)   1
escola(1180700216,80)   1
escola(1180500104,80)   1
escola(1180500114,120)   1
escola(1180500226,80)   1
escola(1181100139,80)   1
Fração da Sub[11801

Fração da Sub[1180800612], escola[1180800607][120]   10.6477
Fração da Sub[1180800614], escola[1180800607][120]   3.30552
Fração da Sub[1180800615], escola[1180800607][200]   9.91657
Fração da Sub[1180800616], escola[1180800607][200]   8.15006
Fração da Sub[1180900101], escola[1180700304][120]   12.9963
Fração da Sub[1180900102], escola[1180800607][200]   13.0826
Fração da Sub[1180900103], escola[1180700307][80]   17.8818
Fração da Sub[1180900104], escola[1180700307][80]   10.6514
Fração da Sub[1180900105], escola[1181000302][200]   12.713
Fração da Sub[1180900106], escola[1181000302][80]   1.47952
Fração da Sub[1180900107], escola[1181000302][120]   2.8727
Fração da Sub[1180900108], escola[1181000302][120]   0.63119
Fração da Sub[1180900109], escola[1180800607][200]   10.7871
Fração da Sub[1180900110], escola[1180800607][200]   11.8203
Fração da Sub[1180900111], escola[1180800607][80]   4.16743
Fração da Sub[1180900112], escola[1180800607][120]   2.25017
Fração da Sub[1180900114], esc

In [19]:
coordinates_Y[1180700110]

40.56364482490964

In [14]:
len(resultsBgriCapacity)

37

In [15]:
resultsBgriCapacity

['escola(1180600121,80)',
 'escola(1180600121,120)',
 'escola(1180400104,120)',
 'escola(1181100114,120)',
 'escola(1180700110,120)',
 'escola(1180300206,80)',
 'escola(1180300206,120)',
 'escola(1180200202,80)',
 'escola(1180800605,80)',
 'escola(1180800605,120)',
 'escola(1180800605,200)',
 'escola(1180100209,80)',
 'escola(1180100209,120)',
 'escola(1180400606,80)',
 'escola(1180400210,80)',
 'escola(1180400210,200)',
 'escola(1180100112,80)',
 'escola(1180100112,120)',
 'escola(1180700304,120)',
 'escola(1180500217,80)',
 'escola(1180700115,120)',
 'escola(1181000302,80)',
 'escola(1181000302,120)',
 'escola(1181000302,200)',
 'escola(1180200225,80)',
 'escola(1180600120,120)',
 'escola(1180800607,80)',
 'escola(1180800607,120)',
 'escola(1180800607,200)',
 'escola(1180700307,80)',
 'escola(1180400603,80)',
 'escola(1180700412,120)',
 'escola(1180700216,80)',
 'escola(1180500104,80)',
 'escola(1180500114,120)',
 'escola(1180500226,80)',
 'escola(1181100139,80)']

In [16]:
# m = None

In [17]:
# disposeDefaultEnv()

Freed default Gurobi environment


# 